In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15))
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

![](kc.PNG "LYMNP 로고")
```
우리나라는 국토의 63.2%가 산림으로 이루어져 있습니다. 등산을 해본 경험이 있다면, 산을 오를 수록 그 주위를 이루고 있는 식물, 나무들이 약간씩 다르다는 걸 느낄 수 있습니다. 과연 어떤 조건에 의해서 특정 식물들이 자랄 수 있는 환경이 만들어 진 것일까요? 그걸 알 수 있다면 어떤 지형의 조건이 주어졌을 때, 그곳에 서식하고 있는 숲의 종류를 알 수 있지 않을까요?

이번 컴퍼티션은 콜로라도 주에 있는 산림의 지형적 특징, 흙의 종류, 지표수까지의 위치 등의 정보를 통해 이 지표면에 서식하는 식생(식물)이 어떤 종류인지 예측하는 대회입니다.
```

# 평가 방법

```
이번 컴퍼티션의 평가 방식은 Mean F1-Score입니다. 설명은 아래와 같습니다.

The evaluation metric for this competition is Mean F1-Score. The F1 score, commonly used in information retrieval, measures accuracy using the statistics precision ( \text{p} ) and recall ( \text{r} ).

Precision is the ratio of true positives (\text{tp}) to all predicted positives (\text{tp} + \text{fp}). Recall is the ratio of true positives (\text{tp}) to all actual positives (\text{tp} + \text{fn}). The F1 score is given by:

[ \text{F1} = 2\frac{\text{p} \cdot \text{r}}{\text{p}+\text{r}}\ \ \mathrm{where}\ \ \text{p} = \frac{\text{tp}}{\text{tp}+\text{fp}},\ \ \text{r} = \frac{\text{tp}}{\text{tp}+\text{fn}} ]

The F1 metric weights recall and precision equally, and a good retrieval algorithm will maximize both precision and recall simultaneously. Thus, moderately good performance on both will be favored over extremely good performance on one and poor performance on the other.

The file should contain a header and have the following format:

id,Cover_Type
0,1  
0,2   
0,4   



```

# 데이터
```
Data Fields
Elevation - 높이 (m)
Aspect - 방향 (표면이 향하는 방향)
Slope - 경사도
Horizontal_Distance_To_Hydrology - 지표수까지의 가장 가까운 수평거리
Vertical_Distance_To_Hydrology - 지표수까지의 가장 가까운 수직거리
Horizontal_Distance_To_Roadways - 차도까지의 가장 가까운 수평거리
Horizontal_Distance_To_Fire_Points - 불이 났던 지점으로부터 가장 가까운 수평거리
Hillshade_9am - 하지점(Summer solstice) 기준으로 9am의 태양의 위치에 따른 음영
Hillshade_Noon - 하지점(Summer solstice) 기준으로 12am의 태양의 위치에 따른 음영
Hillshade_3pm - 하지점(Summer solstice) 기준으로 3pm의 태양의 위치에 따른 음영
Wilderness_Area - 야생 지역 위치 (아래 참고)
Soil_Type - 토양 타입 (아래 참고)
Cover_Type - 산림 타입 (Target) (아래 참고)
Forest Cover Type 별 설명
Spruce/Fir wiki
Lodgepole Pine wiki
Ponderosa Pine wiki
Krummholz wiki
Wilderness Area Type 별 설명 (데이터는 0 ~ 3로 표기)
Rawah Wilderness Area
Neota Wilderness Area
Comanche Peak Wilderness Area
Cache la Poudre Wilderness Area
Soil Type 별 설명 (데이터는 0 ~ 39로 표기 )
1 Cathedral family - Rock outcrop complex, extremely stony.
2 Vanet - Ratake families complex, very stony.
3 Haploborolis - Rock outcrop complex, rubbly.
4 Ratake family - Rock outcrop complex, rubbly.
5 Vanet family - Rock outcrop complex complex, rubbly.
6 Vanet - Wetmore families - Rock outcrop complex, stony.
7 Gothic family.
8 Supervisor - Limber families complex.
9 Troutville family, very stony.
10 Bullwark - Catamount families - Rock outcrop complex, rubbly.
11 Bullwark - Catamount families - Rock land complex, rubbly.
12 Legault family - Rock land complex, stony.
13 Catamount family - Rock land - Bullwark family complex, rubbly.
14 Pachic Argiborolis - Aquolis complex.
15 unspecified in the USFS Soil and ELU Survey.
16 Cryaquolis - Cryoborolis complex.
17 Gateview family - Cryaquolis complex.
18 Rogert family, very stony.
19 Typic Cryaquolis - Borohemists complex.
20 Typic Cryaquepts - Typic Cryaquolls complex.
21 Typic Cryaquolls - Leighcan family, till substratum complex.
22 Leighcan family, till substratum, extremely bouldery.
23 Leighcan family, till substratum - Typic Cryaquolls complex.
24 Leighcan family, extremely stony.
25 Leighcan family, warm, extremely stony.
26 Granile - Catamount families complex, very stony.
27 Leighcan family, warm - Rock outcrop complex, extremely stony.
28 Leighcan family - Rock outcrop complex, extremely stony.
29 Como - Legault families complex, extremely stony.
30 Como family - Rock land - Legault family complex, extremely stony.
31 Leighcan - Catamount families complex, extremely stony.
32 Catamount family - Rock outcrop - Leighcan family complex, extremely stony.
33 Leighcan - Catamount families - Rock outcrop complex, extremely stony.
34 Cryorthents - Rock land complex, extremely stony.
35 Cryumbrepts - Rock outcrop - Cryaquepts complex.
36 Bross family - Rock land - Cryumbrepts complex, extremely stony.
37 Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.
38 Leighcan - Moran families - Cryaquolls complex, extremely stony.
39 Moran family - Cryorthents - Leighcan family complex, extremely stony.
40 Moran family - Cryorthents - Rock land complex, extremely stony.
```



# 파일
```
train.csv - 훈련 셋
test.csv - 테스트 셋
sample_submission.csv - 제출 파일 포맷
```

# 데이터 로드

In [2]:
train = pd.read_csv('soil/train.csv' ) 
test = pd.read_csv('soil/test.csv' ) 
train

,id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area,Cover_Type
0,0,2983,339,23,97.0,NaN,1441.0,223.0,205.0,105.0,3487.0,21,0,1
1,1,3046,375,5,69.0,-3.0,865.0,223.0,225.0,149.0,1522.0,0,2,2
2,2,2540,199,10,128.0,2.0,1009.0,216.0,245.0,146.0,1977.0,0,0,2
3,3,2893,351,21,54.0,19.0,4886.0,247.0,195.0,127.0,1777.0,18,2,2
4,4,2529,58,35,92.0,42.0,455.0,225.0,249.0,168.0,264.0,25,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826219,826219,2626,144,17,60.0,-3.0,1001.0,220.0,257.0,177.0,369.0,25,0,2
826220,826220,3147,47,27,38.0,24.0,1393.0,204.0,224.0,68.0,797.0,0,2,1
826221,826221,3362,203,8,506.0,0.0,802.0,182.0,220.0,116.0,618.0,0,1,1
826222,826222,3127,271,18,127.0,133.0,NaN,192.0,228.0,147.0,NaN,27,2,1


In [3]:
# y 설정
y = train['Cover_Type']
# y_test = test['Cover_Type']
del(train['Cover_Type'])

In [4]:
# 쓸모없는 인덱스 제거
test.drop('id',axis=1 , inplace=True)
train.drop('id',axis=1 , inplace=True)

In [5]:
# 카테고리컬 컬럼, 수치형 컬럼 분류
cat_columns =  ['Soil_Type' ,'Wilderness_Area']
num_columns = [c for c    in train.columns if (c not in cat_columns)]
print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['Soil_Type', 'Wilderness_Area']

Numeric Columns: 
['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']


In [6]:
from sklearn.preprocessing import MinMaxScaler
# 스케일링

scaler = MinMaxScaler()
scaler2 = MinMaxScaler()

scaler.fit(train[num_columns])
scaler2.fit(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']])

MinMaxScaler()

In [7]:
scaler.transform(train[num_columns])

array([[0.47241923, 0.84862385, 0.39393939, ..., 0.66816143, 0.48447205,
        0.45376526],
       [0.49724192, 0.93119266, 0.12121212, ..., 0.75784753, 0.62111801,
        0.21624562],
       [0.29787234, 0.52752294, 0.1969697 , ..., 0.84753363, 0.61180124,
        0.27124381],
       ...,
       [0.62174941, 0.53669725, 0.16666667, ..., 0.73542601, 0.51863354,
        0.1069745 ],
       [0.52915682, 0.69266055, 0.31818182, ..., 0.77130045, 0.61490683,
               nan],
       [0.55200946, 0.11697248, 0.3030303 , ..., 0.73542601, 0.54037267,
        0.58588178]])

In [9]:
scaler2.fit_transform(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']])

array([[0.51432998, 0.18202765, 0.53125   , ..., 0.80365297, 0.43302181,
        0.19639181],
       [0.3140976 , 0.47926267, 0.328125  , ..., 0.82191781, 0.52024922,
        0.18125681],
       [0.66924864, 0.16820276, 0.03125   , ..., 0.74885845, 0.73208723,
        0.53275215],
       ...,
       [0.3954299 , 0.57142857, 0.46875   , ..., 0.83105023, 0.65732087,
        0.06029786],
       [0.41169636, 0.48617512, 0.328125  , ..., 0.75342466, 0.73208723,
        0.14711224],
       [0.58326878, 0.30645161, 0.15625   , ..., 0.84474886, 0.5576324 ,
        0.35452234]])

In [10]:
scaled_data = pd.DataFrame(scaler.transform(train[num_columns]), columns=num_columns)
scaled_data2 = pd.DataFrame(scaler2.transform(test[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']]) ,columns=['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points'])

In [11]:
from sklearn.model_selection import train_test_split
# Train Test 분리
X_train, X_validation, y_train, y_validation = train_test_split(scaled_data, y, train_size=0.75, random_state=42)

In [12]:
from sklearn.model_selection import train_test_split
# Train Test 분리
X_train, X_validation, y_train, y_validation = train_test_split(train, y, train_size=0.75, random_state=42)

# 캣 부스트 사용
```
GBM의 치명적인 문제점 중 하나로 과적합 문제가 존재한다. 
CatBoost는 이 과적합 문제를 해결하면서 동시에 기존 GBM 계열 알고리즘인 
XGBoost, LightGBM 알고리즘보다 학습 속도를 개선하는 장점
 결측치가 매우 많은 데이터셋에는 부적합한 모델


```

In [13]:
from catboost import CatBoostClassifier, Pool, cv



In [14]:
# 데이터 확인
X_train, X_validation, y_train, y_validation

(        Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
 457026       2749      46     41                             581.0   
 647971       3197      68     12                             102.0   
 540916       2810     380      7                             293.0   
 479225       2779     188      4                             209.0   
 716814       2780      60      7                             118.0   
 ...           ...     ...    ...                               ...   
 259178       3366       2      7                             666.0   
 365838       3216     181     11                              36.0   
 131932       2590     130     22                             595.0   
 671155       3332     276     10                              54.0   
 121958       2687      24      9                             196.0   
 
         Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
 457026                            40.0                            280

In [15]:
# y train 인코딩
pd.get_dummies( y_train)

,1,2,3,4
457026,0,1,0,0
647971,1,0,0,0
540916,0,1,0,0
479225,0,1,0,0
716814,0,1,0,0
...,...,...,...,...
259178,1,0,0,0
365838,1,0,0,0
131932,0,0,1,0
671155,1,0,0,0


In [16]:
from sklearn.metrics import accuracy_score, log_loss, make_scorer

In [19]:
import catboost as cb
cb_dtrain = cb.Pool(data = X_train, label = y_train) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.1, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'task_type' : 'GPU',
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param ) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(X_validation), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.4f" % (accuracy_score(y_validation, cb_model_predict) * 100), "%") # 정확도 % 계산

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.8841186	total: 32.9ms	remaining: 3.26s
1:	learn: 0.8849771	total: 65ms	remaining: 3.18s
2:	learn: 0.8911772	total: 97.2ms	remaining: 3.14s
3:	learn: 0.8953827	total: 129ms	remaining: 3.11s
4:	learn: 0.8966834	total: 161ms	remaining: 3.06s
5:	learn: 0.8976807	total: 193ms	remaining: 3.02s
6:	learn: 0.8981664	total: 224ms	remaining: 2.98s
7:	learn: 0.8977533	total: 257ms	remaining: 2.96s
8:	learn: 0.8979405	total: 290ms	remaining: 2.93s
9:	learn: 0.8983617	total: 320ms	remaining: 2.88s
10:	learn: 0.8987264	total: 353ms	remaining: 2.86s
11:	learn: 0.8997963	total: 387ms	remaining: 2.84s
12:	learn: 0.9018168	total: 426ms	remaining: 2.85s
13:	learn: 0.9025882	total: 460ms	remaining: 2.83s
14:	learn: 0.9031239	total: 497ms	remaining: 2.82s
15:	learn: 0.9041600	total: 535ms	remaining: 2.81s
16:	learn: 0.9042697	total: 568ms	remaining: 2.77s
17:	learn: 0.9052509	total: 604ms	remaining: 2.75s
18:	learn: 0.9059270	total: 637ms	remaining: 2.71s
19:	learn: 0.9068404	total: 668ms	remain

파라미터를 임의로 설정 => 정확도 92.7521%
 
 
 정확도 개선 위해 그리드 서치 진행